In [ ]:
# importing lib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# loading the dataset
info = pd.read_csv('INFY.csv')

In [ ]:
info.head()

In [ ]:
info.describe()

In [ ]:
info.Series.unique()

In [ ]:
info = info[info['Series']== 'EQ']

In [ ]:
info.Series.unique()

In [ ]:
info.describe()

In [ ]:
info.head()


In [ ]:
info.tail()

In [ ]:
info.plot(subplots=True,figsize=(15,15));

In [ ]:
# dataframe with only last 90 days
last_90_info =info[404:]
last_90_info.shape

In [ ]:
last_90_info.columns

In [ ]:
# calculating max, mean and min values of price in last 90 days 
last_90_info['Close Price'].max(),last_90_info['Close Price'].mean(),last_90_info['Close Price'].min()

In [ ]:
# checking the format for all columns
info.dtypes

In [ ]:
# changing the date format for further convenience
info['Date']=info['Date'].astype('datetime64[ns]')
info.dtypes

In [ ]:
# trying operations on the new date format
info['Date'][294] - info['Date'][0]

In [ ]:
# creating year and month columns
info['Year']=pd.DatetimeIndex(info['Date']).year
info['month']=pd.DatetimeIndex(info['Date']).month

In [ ]:
info

In [ ]:
# creating a function to compute the volume weighted average price 
def vwap(df):
    q = df['No. of Trades'].values
    p = df['Close Price'].values
    return df.assign(vwap=(p * q).cumsum() / q.cumsum())


In [ ]:
info_new = vwap(info)
info_new

In [ ]:
# creating new dataframe with vwap column
info_new = vwap(info)

# Grouping it by year and month and taking the mean to get the monthly volume weighted average.
info_new.groupby(['Year','month']).mean()

In [ ]:
# creating a new column to having values of daily percentage change  
info_new['Day_Perc_Change'] = info_new["Close Price"].pct_change()*100


In [ ]:
# filling the null value to zero
info_new['Day_Perc_Change']=info_new['Day_Perc_Change'].fillna(0)
info_new

In [ ]:
# Creating fuction to calculate average price for last N days
def avg_price_ndays(df,N):
    data = df.tail(N)
    avg_cal = data['Close Price'].mean()
    print (f'The average price for last {N} days = {avg_cal}')

# Creating fuction to compute Profit/Loss percentage for last N days 
def profit_loss_ndays(df,N):
    data = df.tail(N)
    percentage = (data['Day_Perc_Change'].sum())
    print(f'The profit/loss over the last {N} days = {percentage}%')
    

In [ ]:
# Runing both function for 1 week, 2 week, 1 month, 3 months,6 months and a year
avg_price_ndays(info_new,7)
profit_loss_ndays(info_new,7)

avg_price_ndays(info_new,14)
profit_loss_ndays(info_new,14)

avg_price_ndays(info_new,30)
profit_loss_ndays(info_new,30)

avg_price_ndays(info_new,90)
profit_loss_ndays(info_new,90)

avg_price_ndays(info_new,180)
profit_loss_ndays(info_new,180)
                 
avg_price_ndays(info_new,365)
profit_loss_ndays(info_new,365)

In [ ]:
# creating a function to help create Trend column
def trend_col_creator(df):
    data =df['Day_Perc_Change']
    length =len(df)
    trend = []
    for i in data:
        if -0.5<i<0.5:
            trend.append('Slightly or NO Change')
        elif 0.5<i<1:
            trend.append('Slightly Positive')
        elif -1<i<-0.5:
            trend.append('Slightly Negative')
        elif 1<i<3:
            trend.append('Positive')
        elif -3<i<-1:
            trend.append('Negative') 
        elif 3<i<7:
            trend.append('Among Top Gainers')
        elif -7<i<-3:
            trend.append('Among Top Losers')  
        elif i<7:
            trend.append('Bull Run')
        elif i>7:
            trend.append('Bear drop')
    print(len(trend))
    df['trend'] = trend
    return df['trend']

In [ ]:
trend_col_creator(info_new)


In [ ]:
# checking the category types within the Trend column
info_new['trend'].unique()

In [ ]:
# Calculating the mean and median of total traded quantity over different trends 
info['Total Traded Quantity'].groupby(info_new['trend']).mean()


In [ ]:
info['Total Traded Quantity'].groupby(info_new['trend']).median()

In [ ]:
# final dataframe
info_new.head()

In [ ]:
# Saving the dataframe into a csv file 
info_new.to_csv('week2.csv')